In [133]:
import numpy as np
import pandas as pd 
import torch

In [134]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\sande\.cache\kagglehub\datasets\uciml\pima-indians-diabetes-database\versions\1


In [135]:
import os 

print(os.listdir(path))

['diabetes.csv']


In [136]:
df=pd.read_csv(os.path.join(path,"diabetes.csv"))

In [137]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [139]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [140]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [141]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [142]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (614, 8)
X_test shape: (154, 8)
y_train shape: (614,)
y_test shape: (154,)


In [143]:
#Sccaling 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [144]:
X_train.shape

(614, 8)

In [145]:
y_train.shape

(614,)

In [146]:
X_train.dtype

dtype('float64')

In [147]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)

y_train_tensor = torch.from_numpy(y_train.values).long()
y_test_tensor  = torch.from_numpy(y_test.values).long()



Start model buliding

In [148]:
class My_Network():
    def __init__(self,x):
        self.weights = torch.randn(x.shape[1],dtype=torch.float64,requires_grad=True)
        self.bias = torch.randn(1,dtype=torch.float64,requires_grad=True)
    
    def forward(self,x): 
        z=torch.matmul(x,self.weights)+self.bias
        y_pred=torch.relu(z)
        return y_pred

    def loss_function(self, y_pred, y):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss
        

In [149]:
#parameters
learning_rate = 0.01
epochs = 10000


model = My_Network(X_train_tensor)

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    loss = model.loss_function(y_pred, y_train_tensor)
    loss.backward()
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
    

Epoch 1/10000, Loss: 7.602560138239172
Epoch 2/10000, Loss: 7.4659031964319515
Epoch 3/10000, Loss: 7.311182563493999
Epoch 4/10000, Loss: 7.02961818547121
Epoch 5/10000, Loss: 6.995689522726406
Epoch 6/10000, Loss: 6.97431939329485
Epoch 7/10000, Loss: 6.674689063992974
Epoch 8/10000, Loss: 6.63620480615184
Epoch 9/10000, Loss: 6.545582397651316
Epoch 10/10000, Loss: 6.5392841369168835
Epoch 11/10000, Loss: 6.545297968619532
Epoch 12/10000, Loss: 6.517213983540475
Epoch 13/10000, Loss: 6.494308565307011
Epoch 14/10000, Loss: 6.638409523708215
Epoch 15/10000, Loss: 6.583714869691226
Epoch 16/10000, Loss: 6.546788276013907
Epoch 17/10000, Loss: 6.535691608202459
Epoch 18/10000, Loss: 6.528589362409186
Epoch 19/10000, Loss: 6.503354078188469
Epoch 20/10000, Loss: 6.477038850639889
Epoch 21/10000, Loss: 6.468329798411062
Epoch 22/10000, Loss: 6.445052993984475
Epoch 23/10000, Loss: 6.422050350194158
Epoch 24/10000, Loss: 6.285427255732505
Epoch 25/10000, Loss: 6.253232402382901
Epoch 26/1

In [150]:
#model Evaluation

with torch.inference_mode():
    y_pred_test=model.forward(X_test_tensor)
    y_pred=(y_pred_test>0.5).float()
    accuracy=(y_pred==y_test_tensor).float().mean()
    print("Accuracy on test set: ",accuracy.item())


Accuracy on test set:  0.7857142686843872
